# visulize the visual mesh and collision mesh for scanned rabbit

In [2]:
import sys
sys.path.append(".")
from tools.visualization_utils import *
import numpy as np
# import open3d as o3d
import trimesh

## voxel resolution = 32

In [ ]:
mesh = o3d_fem_mesh("/media/aurmr/data1/weird/IsaacLab/logs/rl_games/franka_lift/deform_fem/0/0.obj")

raw_vertices = trimesh.load("/media/aurmr/data1/weird/IsaacLab/source/assets/Plush/usd/rabbit/rabbit.obj").vertices/100*0.6

translation = np.array([0.47, -0.0, (np.max(raw_vertices,0)[1]-np.min(raw_vertices,0)[1])/2])
quat = obtain_target_cam_quat([1, 2, 0,1], [-1.57, 1.57, -1.57,1.57])

transformed_vertices = transform_vertices(raw_vertices,quat,translation)
pc = vis_pc(transformed_vertices)
coordinate_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1, origin=[0, 0, 0])
o3d.visualization.draw_geometries([mesh,pc,coordinate_frame], mesh_show_wireframe=True)

# load the static gaussian images from the h5py

In [5]:
import h5py
import cv2
import os

dir_name = "logs/static_gs/cat/raw/Isaac-Lift-DeformCube-Franka-Play-v0"
type = "cat"
os.makedirs(f"{dir_name}/raw_imgs", exist_ok=True)
with h5py.File(f"{dir_name}/{type}.hdf5", 'r') as file:

    rgb_data = file["data"]["demo_0"]["obs"]["rgb"][10:, :, :, :, :3]

    rgb_dataset = rgb_data.reshape(-1, rgb_data.shape[2], rgb_data.shape[3], 3)
    for img_idx in range(rgb_dataset.shape[0]):
        cv2.imwrite(
            f"{dir_name}/raw_imgs/frame_{img_idx:05d}.jpg",
            rgb_dataset[img_idx][..., ::-1],
        )


# convert h5py to image

In [42]:
import h5py
import cv2
import os
import isaaclab.utils.math as math_utils
import numpy as np
from pathlib import Path
import json
import torch
import math


def obtain_target_cam_quat(axis, angles):
    quat_list = []
    for index, cam_axis in enumerate(axis):
        euler_xyz = torch.zeros(3)
        euler_xyz[cam_axis] = angles[index]
        quat_list.append(
            math_utils.quat_from_euler_xyz(euler_xyz[0], euler_xyz[1],
                                           euler_xyz[2]))
    if len(quat_list) == 1:
        return quat_list[0]
    else:
        target_quat = quat_list[0]
        for index in range(len(quat_list) - 1):

            target_quat = math_utils.quat_mul(quat_list[index + 1],
                                              target_quat)
        return target_quat


dir_name = "/media/lme/data4/weird/IsaacLab/logs/static_gs/cat/raw/Isaac-Lift-DeformCube-Franka-Play-v0"
type = "cat"
keep_original_world_coordinate = False
os.makedirs(f"{dir_name}/raw_imgs", exist_ok=True)
with h5py.File(f"{dir_name}/{type}.hdf5", 'r') as file:

    rgb_data = file["data"]["demo_0"]["obs"]["rgb"][10:, :, :, :, :3]
    extrinsic_params = file["data"]["demo_0"]["obs"]["extrinsic_params"][
        10:,
    ]
    intrinsic_params = file["data"]["demo_0"]["obs"]["intrinsic_params"][
        10:,
    ]

    rgb_dataset = rgb_data.reshape(-1, rgb_data.shape[2], rgb_data.shape[3], 3)

    extrinsic_dataset = extrinsic_params.reshape(-1, extrinsic_params.shape[2],
                                                 extrinsic_params.shape[3])

    intrinsic_dataset = intrinsic_params.reshape(-1, intrinsic_params.shape[2],
                                                 intrinsic_params.shape[3])
    frames = []
    out = {}

    intrinsic = intrinsic_dataset[0]
    # import pdb
    # pdb.set_trace()
    out["w"] = rgb_dataset[0].shape[1]
    out["h"] = rgb_dataset[0].shape[0]
    out["fl_x"] = float(intrinsic[0][0])
    out["fl_y"] = float(intrinsic[0][0])
    out["cx"] = float(intrinsic[0][2])
    out["cy"] = float(intrinsic[1][2])
    out["k1"] = float(0.0)
    out["k2"] = float(0.0)
    out["p1"] = float(0.0)
    out["p2"] = float(0.0)
    out["camera_model"] = "OPENCV"
    for img_idx in range(rgb_dataset.shape[0]):
        cv2.imwrite(
            f"{dir_name}/raw_imgs/frame_{img_idx:05d}.jpg",
            rgb_dataset[img_idx][..., ::-1],
        )
        rotate_matrix = np.eye(4)
        rotate_matrix[:3, :3] =math_utils.matrix_from_quat(
        obtain_target_cam_quat([1,2], [3.14,-1.57])).numpy()
        w2c = extrinsic_dataset[img_idx]
        w2c = rotate_matrix @ extrinsic_dataset[img_idx]
        c2w = np.linalg.inv(w2c)

        # c2w[0:3, 1:3] *= -1

        # # if not keep_original_world_coordinate:
        # c2w = c2w[np.array([0, 2, 1, 3]), :]
        # c2w[2, :] *= -1

        # c2w[:3, :3] = rotation_opengl
        name = f"raw_imgs/frame_{img_idx:05d}.jpg"
        name = Path(name)

        frame = {
            "file_path": name.as_posix(),
            "transform_matrix": c2w.tolist(),
            "colmap_im_id": img_idx,
        }

        frames.append(frame)
    out["frames"] = frames
    applied_transform = None

    with open(Path(dir_name) / "transforms.json", "w", encoding="utf-8") as f:
        json.dump(out, f, indent=4)

ModuleNotFoundError: No module named 'Metashape'